In [223]:
# Text Preparation for Error Checker

In [7]:
import nltk
import os
import re


In [ ]:
# Define The Target Directory

In [8]:
os.chdir(r'C:\Users\Chris.Cirelli\Desktop\Python Programming Docs\GSU\Sprint Project\Legal Docs')
Target_dir = os.listdir(r'C:\Users\Chris.Cirelli\Desktop\Python Programming Docs\GSU\Sprint Project\Legal Docs')

In [9]:
Target_dir

['.ipynb_checkpoints',
 'GA_Northern_1_15-cv-04247-TWT_26.txt',
 'GA_Northern_1_15-cv-04247-TWT_32.txt',
 'GA_Northern_1_15-cv-04249-ODE_4.txt',
 'GA_Northern_1_15-cv-04258-AT_0.txt',
 'GA_Northern_1_15-cv-04260-CC_0.txt',
 'GA_Northern_1_15-cv-04260-CC_8.txt',
 'GA_Northern_1_15-cv-04264-AT_0.txt',
 'GA_Northern_1_15-cv-04281-RWS_0.txt',
 'GA_Northern_1_15-cv-04284-WSD_0.txt',
 'GA_Northern_1_15-cv-04285-AT_0.txt',
 'GA_Northern_1_15-cv-04288-AT_0.txt',
 'GA_Northern_1_15-cv-04298-MHC_0.txt',
 'GA_Northern_1_15-cv-04303-ODE_0.txt',
 'GA_Northern_1_15-cv-04310-LMM_0.txt']

In [4]:
# Import Personal Modules

In [179]:
def clean_text_4_classification_remove_backslashes(Text_file):
    '''The purpose of this function is to clean the text files of numerous instances of backslashes 
    in order to prepare them for the regex expression search. 
    
    Input  =   Single text file 
    Output =   Single text file cleaned 
    '''
    
    # Convert text to lowercase
    Text_file_lower = Text_file.lower()
    
    # Split any values in the text on the backslash.  The Text_split_slash should return a list. 
    Text_split_slash = Text_file_lower.split('\\')
        
    # Return the list to a text. 
    Text_rejoined = ' '.join(Text_split_slash)
                
    # Return a list of the cleaned text docs. 
    return Text_rejoined


def clean_text_4_classification_remove_nABC(Text_file):
    '''The purpose of this function is to remove the 'n' that appears before words that begin with an upper case letter.  
    Input  =   Single txt file
    Output =   Clean list of tokens from original txt file
   
    '''
    # Define the regex expression that you want to search for. 
    Regex_exp = re.compile('n[A-Z*]')
    
    # Create a list to capture the tokens once they are cleaned 
    Text_tokenized_cleaned = []
            
    # Tokenize the given text
    Text_tokenized = nltk.word_tokenize(Text_file)
            
    # Run for loop over tokens for a given text. 
    for token in Text_tokenized:

        # Search for the regex expression
        Regex_search = re.search(Regex_exp, token)
                
        # Test if there was match (None = no match)
        if Regex_search != None:
                     
            # If there was a match, take all letters after the 'n'.   
            token_cleaned = token[1:]
                    
            Text_tokenized_cleaned.append(token_cleaned)
                        
        # If the Regex_search returned None, return the token back to the Text_tokenized_cleaned list
        else:
            Text_tokenized_cleaned.append(token)
    
    # Return a list of clean tokens
    return Text_tokenized_cleaned


def create_dict_punct():
    '''The purpose of this function is to simply create a dictionary of punctuation symbols to use
    in other functions'''
    import string
    Dict = {}
    Punct = string.punctuation
    for x in Punct:
        Dict[x] = ''
    return Dict 

def strip_punctuation(Token_list):
    '''The purpose of this function is to strip the punctuation from a list of tokens. 
    Input  =  List of tokens
    Output =  List of tokens absent punctuation.  
    '''
    
    # Import punctuation dictionary
    Dict_punct = create_dict_punct()

    # Create a list to capture the cleaned tokens
    Clean_token_list = []    
        
    # Iterate over the tokens in the txt file
    for x in Token_list:
        if x not in Dict_punct:
            # Append tokens to clean token list
            Clean_token_list.append(x)
    
    # Return a list of cleaned text
    return Clean_token_list

def strip_two_letter_words(Token_list):
    '''The purpose of this function is to remove any two letter tokens from a list of tokens.
    Input  =   List of tokens
    Output =   List of tokens absent two letter words'''
    
    List = [x for x in Token_list if len(x) > 2]
    
    return List

def create_dict_stopwords():
    from nltk.corpus import stopwords
    Stopwords = stopwords.words('english')                  
    Dict = {}
    for x in Stopwords:
        Dict[x] = ''
    return Dict

def strip_stop_words(Token_list):
    stop_words = create_dict_stopwords()
    List = []
    for x in Token_list:
        if x not in stop_words:
            List.append(x)
    return List


In [112]:
# Text Cleaning Pipeline

In [186]:
'''This pipeline will be placed inside a larger function that loops over the Target Directory, identifies the text files,
    opens them, etc, and also captures the target file, tokenized text and statistics.  We'll need to create these 
    variables within the master function. 
'''

def text_clearning_pipeline_Input_4_Error_Checker_Function(Text_file):
    '''The purpose of this function is to prepare text for use with the Error Checker Program
    Input  =  Single text file
    Output =  List of clean tokens representing a single text. 
    '''
    
    # Run Clearning Pipeline
    txt_strip_backslashes = clean_text_4_classification_remove_backslashes(Text_file)
    txt_strip_nABC = clean_text_4_classification_remove_nABC(txt_strip_backslashes)
    txt_strip_punct = strip_punctuation(txt_strip_nABC)
    txt_strip_2_letter_words = strip_two_letter_words(txt_strip_punct)
    txt_strip_stop_words = strip_stop_words(txt_strip_2_letter_words)
        
    # Return List of clean tokenized text
    return txt_strip_stop_words
    

In [106]:
# Mock Final Function Setup to test Clearning Pipeliine Pipeline (mock)

In [417]:
File = Target_dir[1]
File_open = open(File, 'rb')
Text_bytes = File_open.read()
Text_str = str(Text_bytes)
Text_tokens_cleaned = text_clearning_pipeline_Input_4_Error_Checker_Function(Text_str)

In [421]:
# Test Cleaning Pipeline
Text_tokens_cleaned[:10]

["b'case",
 '1:15-cv-04247-twt',
 'document',
 'filed',
 '10/24/16',
 'page',
 'nin',
 'united',
 'states',
 'district']

In [ ]:
# ERROR CHECKER FUNCTION

In [29]:
import os
import nltk
from nltk.corpus import wordnet as wn
import sys

In [109]:
# Create Set of Wordnet Dictionary

In [41]:
def create_Set_wordnet_dict():
    '''The purpose of this function is to create a set of all words from the wordnet dictionary.
    Input  = None
    Output = Set object of all words. 
    '''
    # Import words from wordnet
    from nltk.corpus import wordnet as wn
    Words = wn.words()

    # Create List to capture words  
    List_dict_words = []; [List_dict_words.append(x) for x in Words]
    
    # Create Set
    Set_dict_words = set(List_dict_words)
    
    # Return Set
    return Set_dict_words

In [ ]:
# Create Error Counting Function

In [102]:
def Error_Percentage_Per_Document(Tokenized_text_list_tokens):
    '''The purpose of this function is to count the tokens in a given text that do not match a word in the Wordnet Dict
    Input  = List of tokens
    Ouput  = Float that represents the percentage of error in the documents  
    
    Note:  the final function will need to map the error percentage to the document name. 
    '''
    
    # Create Set of dictionary words
    Word_set = create_Set_wordnet_dict()
    
    # Create Counter & Calc Num of total tokens 
    Num_tokens = len(Tokenized_text_list_tokens)
    Count_false = 0
    
    # Loop over list to unpack tokenz
    for token in Tokenized_text_list_tokens:
        
    # Check to see if token in Word_set
        if token not in Word_set:
            # if token in set
            Count_false += 1
        
    # Error percentage is the Num of Errors / Total tokens * 100. 
    Error_percentage = round(((Count_false / Num_tokens)*100), 2)
    
    # Return Error_percentage_value 
    return Error_percentage
    

In [189]:
Error_Percentage_Per_Document(Text_tokens_cleaned)

51.58

In [ ]:
# Create Error Frequency Distribution 

In [412]:
def Error_Frequency_Distribution(Tokenized_text_list_tokens, Obj_is_list_of_lists = '', Graph = ''):
    '''The purpose of this function is generate a frequency distribution of errors for as many tokenized text files as are
    passed to the function.  
    Input  = Tokenized text. If a object passed is a list of text, set List_of_texts to True.  Otherwise, defaults to False
    Ouput  = Dictionary whose 'key' is the error and 'values' is the   
    '''
        
    # Create Set of dictionary words
    Word_set = create_Set_wordnet_dict()
    
    # Create Dictionary to Capture Errors (keys) and Count (values)
    Error_Freq_Dist_dict = {}
    
    # Check if List_of_texts True
    if Obj_is_list_of_lists is False:
    
        # Loop over list to unpack tokens
        for token in Tokenized_text_list_tokens:
            
            # Check to see if token in Word_set
            if token not in Word_set:
                Error_Freq_Dist_dict[token] = Error_Freq_Dist_dict.get(token, 0) +1
    else:
        # Loop over list of texts:
        for token_list in Tokenized_text_list_tokens:
            
            # Loop over list to unpack tokens
            for token in token_list:
            
                # Check to see if token in Word_set
                if token not in Word_set:
                    
                    Error_Freq_Dist_dict[token] = Error_Freq_Dist_dict.get(token, 0) +1

    # Create Pandas Dataframe
    import pandas as pd, matplotlib.pyplot as plt
    Index = [1]
    df = pd.DataFrame(Error_Freq_Dist_dict, index = Index)
    df_tp = pd.DataFrame.transpose(df)
    df_sorted = df_tp.sort_values(1, ascending = False)
                    
    if Graph == True:
        graph = df_sorted.plot(kind = 'bar')
        return plt.show()
    
    else:
        # Return Error_percentage_value 
        return df_sorted
    

In [416]:
Error_Frequency_Distribution(List_clean_txt_tokens, Obj_is_list_of_lists= True, Graph = False)

,1
x80,60
xe2,60
x99s,45
nthe,30
states,30
xef,30
xac,30
2016.,30
norder,30
x94cv-04247-,15


In [ ]:
# Defining the sub functions of the error checker

In [ ]:
'''
Single document:

1. Count of errors
2. List of enumerated errors
3. Len of document
4. % of Error of Document
5. Document type?
6. Write to Excel. 

Statistics For All Docs (Separate Function)
1. Frequency distribution of errors across all documents
2. % Error by document type
3. Ranking of documents by highest to lowest quality
4. Standard deviation of document (will require mean of all docs). 


'''

In [ ]:
#### FINAL FUNCTION 1

In [33]:
'''Obtain the metrics we are looking for for only one text.  Then we can run the larger pipeliine function over a
directory and capture the values in a dataframe'''

def Error_checker(Text_File):
    '''The purpose of this function is to identify 
    a.) The % of errors in a text files translated from a pdf using tesseract OCR
    b.) The frequency of the errors in any given text or series of text'''
    
    # Clearning Pipeline
    text_clearning_pipeline_Input_4_Error_Checker_Function(None)
    
    # Error detection pipeline
    
    return None
    